In [1]:
import pandas as pd
from flot_util import *
from flot_visualization import *
from flot_prediction import *
from flot_feature_engineering import *

In [2]:
df = pd.read_csv(r'C:\Users\rcpsi\OneDrive\Documents\langchain\Flotation\dados\Flotacao_Dados_Final.csv',sep=';')

## Data cleaning

In [3]:
df = parse_time(df)
df = remove_missing(df)
df = filter_by_ph(df)
df = filter_by_flow(df)
df_clean = drop_consecutive_duplicates_tolerance(df)

#df_clean = add_time_features(df_clean)
#df_clean = add_ph_features(df_clean)
#df_clean = add_dosage_flow_features(df_clean)
#df_clean = add_cell_level_stats(df_clean)
df_clean = add_rolling_features(df_clean)

#df_clean = add_delta_silica_regular(df_clean)

In [4]:
df_clean.shape

(2618, 32)

In [5]:
df_clean.head()

,inicio,fim,conc_silica,conc_fe,param_dosagem_amido,dosagem_amina_conc_magnetica,ph_flotacao_linha01,ph_flotacao_linha02,densidade_alimentacao_flotacao,nivel_celula_li640101,...,operacao,duration_min,dosagem_amina_conc_magnetica_rollmean_3,vazao_alimentacao_flotacao_rollmean_3,densidade_alimentacao_flotacao_rollmean_3,param_dosagem_amido_rollmean_3,dosagem_amina_conc_magnetica_rollmean_5,vazao_alimentacao_flotacao_rollmean_5,densidade_alimentacao_flotacao_rollmean_5,param_dosagem_amido_rollmean_5
0,2024-02-27 00:00:00,2024-02-27 00:20:00,2.72,64.59,492.50,152.54,9.47,7.99,1.60,369.44,...,True,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-02-27 01:00:00,2024-02-27 01:20:00,1.23,67.40,492.50,147.58,9.60,7.95,1.57,369.84,...,True,20.0,152.540000,907.140000,1.600,492.500000,152.5400,907.140000,1.6000,492.500000
2,2024-02-27 03:00:00,2024-02-27 03:20:00,1.93,65.09,491.67,135.00,9.50,7.92,1.60,368.38,...,True,20.0,150.060000,792.560000,1.585,492.500000,150.0600,792.560000,1.5850,492.500000
3,2024-02-27 05:00:00,2024-02-27 05:20:00,2.04,65.46,491.67,146.53,9.38,9.50,1.60,369.72,...,True,20.0,145.040000,811.103333,1.590,492.223333,145.0400,811.103333,1.5900,492.223333
4,2024-02-27 07:00:00,2024-02-27 07:20:00,4.56,63.75,491.67,150.75,9.43,9.52,1.60,370.38,...,True,20.0,143.036667,800.046667,1.590,491.946667,145.4125,826.820000,1.5925,492.085000


## Exploratory data analysis

In [6]:
plot_time_series_with_gaps(df_clean,'inicio','conc_silica')

In [7]:
# Usage examples:
# plot_time_series(df_clean, 'inicio', 'conc_silica')
# plot_histogram(df_clean, 'conc_silica')
plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')
# plot_pairwise(df_clean, ['conc_silica', 'conc_fe', 'dosagem_amina_conc_magnetica', 'vazao_alimentacao_flotacao'])

In [8]:
# df_clean.columns

In [9]:
plot_scatter(df_clean, 'ph_flotacao_linha01', 'conc_silica')

In [10]:
plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')

## Prediction

In [11]:
df_clean.columns

Index(['inicio', 'fim', 'conc_silica', 'conc_fe', 'param_dosagem_amido',
       'dosagem_amina_conc_magnetica', 'ph_flotacao_linha01',
       'ph_flotacao_linha02', 'densidade_alimentacao_flotacao',
       'nivel_celula_li640101', 'TO_LI6401_02', 'nivel_celula_li640201',
       'nivel_celula_li640202', 'nivel_celula_li641101',
       'nivel_celula_li641102', 'nivel_celula_li641201',
       'nivel_celula_li641202', 'nivel_celula_li642101',
       'nivel_celula_li642201', 'nivel_celula_li643101',
       'nivel_celula_li643201', 'vazao_alimentacao_flotacao', 'operacao',
       'duration_min', 'dosagem_amina_conc_magnetica_rollmean_3',
       'vazao_alimentacao_flotacao_rollmean_3',
       'densidade_alimentacao_flotacao_rollmean_3',
       'param_dosagem_amido_rollmean_3',
       'dosagem_amina_conc_magnetica_rollmean_5',
       'vazao_alimentacao_flotacao_rollmean_5',
       'densidade_alimentacao_flotacao_rollmean_5',
       'param_dosagem_amido_rollmean_5'],
      dtype='object')

In [12]:
features = df_clean.columns.tolist()
drops = ['inicio','fim','conc_fe', 'operacao','duration_min']
target = ['conc_silica']    

features = list(set(features) - set(drops))
features = list(set(features) - set(target))

print(df_clean.shape)
df_clean.dropna(inplace=True)
print(df_clean.shape)
df_clean.drop(columns=drops, inplace=True)


(2618, 32)
(2617, 32)


In [13]:
res = evaluate_models(df_clean, features, target=target[0])
res

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.617571,2.931900,0.070272,74.620326
1,RandomForest,1.502971,2.779389,0.164481,68.621512
2,XGBoost,1.725851,3.036623,0.002668,80.794782


In [14]:
#res_weights = evaluate_models_weights(df_clean, features, target=target[0])
#res_weights

In [ ]:
res_q = evaluate_models_quantile(df_clean, features, 'conc_silica', show_residuals=False, quantile=0.9)
res_q

TypeError: 'bool' object is not callable